## Import Libraries

In [2]:
import warnings
import pandas as pd

warnings.filterwarnings('ignore')

## Load data
   
   Data source: https://datos.madrid.es/portal/site/egob

In [3]:
# Load file
weather_daily = "add_path_weather_flat_file"
weather_df = pd.read_csv(weather_daily, header = 0, sep = ';')

In [4]:
weather_df.head()

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,D01,V01,D02,...,D27,V27,D28,V28,D29,V29,D30,V30,D31,V31
0,28,79,102,81,28079102_81_98,2019,1,0.66,V,1.16,...,2.57,V,2.93,V,3.23,V,3.18,V,4.72,V
1,28,79,102,81,28079102_81_98,2019,2,4.32,V,2.98,...,1.36,V,0.97,V,0.00,N,0.00,N,0.00,N
2,28,79,102,81,28079102_81_98,2019,3,1.57,V,1.13,...,1.48,V,1.69,V,2.89,V,1.99,V,1.72,V
3,28,79,102,81,28079102_81_98,2019,4,1.32,V,1.21,...,1.00,V,1.82,V,1.97,V,1.84,V,0.00,N
4,28,79,102,81,28079102_81_98,2019,5,2.06,V,2.21,...,1.77,V,3.32,V,3.76,V,3.04,V,1.89,V


## Support Functions

In [8]:
# All months in dataset have 31 days. Function to correct number of days for each month
def filter_days(input_df, col_name_month, col_name_day):

    # Dictionary with the max numb. day for each month year 2019
    numb_days_per_month = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}

    filtered_df = input_df.copy()  # Create a copy of the original DataFrame
    
    filtered_df[col_name_day] = filtered_df[col_name_day].astype(int) # Optional step, weather dataset date is formated as str

    # Iterate over rows of the DataFrame
    for index, row in filtered_df.iterrows():
        month = row[col_name_month]
        day = row[col_name_day]
        
        # Check if the day exceeds the limit for the corresponding month
        if day > numb_days_per_month.get(month, 31):
            filtered_df.drop(index, inplace=True)  # Drop the row if the day is invalid
    
    return filtered_df

## Data Cleaning and Preprocessing

In [5]:
# Transpose day columns to DAY/VALUE column
day_melted_df = pd.melt(weather_df, id_vars=['PROVINCIA','MUNICIPIO','ESTACION', 'MAGNITUD', 'PUNTO_MUESTREO', 'ANO', 'MES'],
                        value_vars=['D{:02d}'.format(day) for day in range(1, 32)],
                        var_name='DAY', value_name='VALUE')

day_melted_df['DAY'] = day_melted_df['DAY'].str.replace('D', '')
day_melted_df

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DAY,VALUE
0,28,79,102,81,28079102_81_98,2019,1,01,0.66
1,28,79,102,81,28079102_81_98,2019,2,01,4.32
2,28,79,102,81,28079102_81_98,2019,3,01,1.57
3,28,79,102,81,28079102_81_98,2019,4,01,1.32
4,28,79,102,81,28079102_81_98,2019,5,01,2.06
...,...,...,...,...,...,...,...,...,...
35025,28,79,59,89,28079059_89_98,2019,8,31,0.00
35026,28,79,59,89,28079059_89_98,2019,9,31,0.00
35027,28,79,59,89,28079059_89_98,2019,10,31,0.40
35028,28,79,59,89,28079059_89_98,2019,11,31,0.00


In [6]:
# Transpose Validate columns to DAY/VALID columns
valid_melted_df = pd.melt(weather_df, id_vars=['PROVINCIA','MUNICIPIO','ESTACION', 'MAGNITUD', 'PUNTO_MUESTREO', 'ANO', 'MES'],
                        value_vars=['V{:02d}'.format(val) for val in range(1, 32)],
                        var_name='DAY',
                        value_name='VALID')


valid_melted_df['DAY'] = valid_melted_df['DAY'].str.replace('V', '')
valid_melted_df

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DAY,VALID
0,28,79,102,81,28079102_81_98,2019,1,01,V
1,28,79,102,81,28079102_81_98,2019,2,01,V
2,28,79,102,81,28079102_81_98,2019,3,01,V
3,28,79,102,81,28079102_81_98,2019,4,01,V
4,28,79,102,81,28079102_81_98,2019,5,01,V
...,...,...,...,...,...,...,...,...,...
35025,28,79,59,89,28079059_89_98,2019,8,31,V
35026,28,79,59,89,28079059_89_98,2019,9,31,N
35027,28,79,59,89,28079059_89_98,2019,10,31,V
35028,28,79,59,89,28079059_89_98,2019,11,31,N


In [7]:
# Merged both dataframes into one:
merged_df = pd.merge(day_melted_df, valid_melted_df[['PROVINCIA','MUNICIPIO','ESTACION', 'MAGNITUD', 'PUNTO_MUESTREO', 'ANO', 'MES', 'DAY','VALID']], 
                     on=['PROVINCIA','MUNICIPIO','ESTACION', 'MAGNITUD', 'PUNTO_MUESTREO', 'ANO', 'MES','DAY'], 
                     how='left')

display(merged_df)

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DAY,VALUE,VALID
0,28,79,102,81,28079102_81_98,2019,1,01,0.66,V
1,28,79,102,81,28079102_81_98,2019,2,01,4.32,V
2,28,79,102,81,28079102_81_98,2019,3,01,1.57,V
3,28,79,102,81,28079102_81_98,2019,4,01,1.32,V
4,28,79,102,81,28079102_81_98,2019,5,01,2.06,V
...,...,...,...,...,...,...,...,...,...,...
35025,28,79,59,89,28079059_89_98,2019,8,31,0.00,V
35026,28,79,59,89,28079059_89_98,2019,9,31,0.00,N
35027,28,79,59,89,28079059_89_98,2019,10,31,0.40,V
35028,28,79,59,89,28079059_89_98,2019,11,31,0.00,N


In [9]:
# Filter out dummy days
correct_weather_df = filter_days(merged_df, 'MES', 'DAY')
display(correct_weather_df.head())
display(correct_weather_df.shape)

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DAY,VALUE,VALID
0,28,79,102,81,28079102_81_98,2019,1,1,0.66,V
1,28,79,102,81,28079102_81_98,2019,2,1,4.32,V
2,28,79,102,81,28079102_81_98,2019,3,1,1.57,V
3,28,79,102,81,28079102_81_98,2019,4,1,1.32,V
4,28,79,102,81,28079102_81_98,2019,5,1,2.06,V


(34359, 10)

In [10]:
# Keep only values from Estacion 24
weather_2019_24 = correct_weather_df[correct_weather_df['ESTACION']==24]
weather_2019_24

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DAY,VALUE,VALID
687,28,79,24,80,28079024_80_98,2019,1,1,6.0,V
688,28,79,24,80,28079024_80_98,2019,2,1,6.0,V
689,28,79,24,80,28079024_80_98,2019,3,1,19.0,V
690,28,79,24,80,28079024_80_98,2019,4,1,14.0,V
691,28,79,24,80,28079024_80_98,2019,5,1,35.0,V
...,...,...,...,...,...,...,...,...,...,...
34668,28,79,24,89,28079024_89_98,2019,5,31,0.0,V
34670,28,79,24,89,28079024_89_98,2019,7,31,0.9,V
34671,28,79,24,89,28079024_89_98,2019,8,31,0.0,V
34673,28,79,24,89,28079024_89_98,2019,10,31,0.3,V


In [15]:
# Following values will be manually imputed by the measurements obtained from ESTACION 56
weather_2019_24[(weather_2019_24['VALID']=='N') & (weather_2019_24['MES']==11) & (~weather_2019_24['MAGNITUD'].isin([80,88]))]

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DAY,VALUE,VALID
15392,28,79,24,81,28079024_81_98,2019,11,14,2.36,N
15404,28,79,24,82,28079024_82_98,2019,11,14,223.00,N
15416,28,79,24,83,28079024_83_98,2019,11,14,7.20,N
15428,28,79,24,86,28079024_86_98,2019,11,14,80.00,N
15440,28,79,24,87,28079024_87_98,2019,11,14,931.00,N
16522,28,79,24,81,28079024_81_98,2019,11,15,0.21,N
16534,28,79,24,82,28079024_82_98,2019,11,15,90.00,N
16546,28,79,24,83,28079024_83_98,2019,11,15,-29.60,N
16558,28,79,24,86,28079024_86_98,2019,11,15,14.00,N
16570,28,79,24,87,28079024_87_98,2019,11,15,859.00,N


In [37]:
weather_2019 = weather_2019_24.pivot_table(index=['PROVINCIA','MUNICIPIO','ESTACION', 'ANO', 'MES', 'DAY'], columns='MAGNITUD', values='VALUE').reset_index()
weather_2019

MAGNITUD,PROVINCIA,MUNICIPIO,ESTACION,ANO,MES,DAY,80,81,82,83,86,87,88,89
0,28,79,24,2019,1,1,6.0,0.69,22.0,4.4,67.0,957.0,101.0,0.0
1,28,79,24,2019,1,2,6.0,0.66,331.0,4.0,69.0,955.0,99.0,0.0
2,28,79,24,2019,1,3,6.0,0.71,115.0,4.4,63.0,954.0,103.0,0.0
3,28,79,24,2019,1,4,5.0,0.68,54.0,3.7,69.0,956.0,102.0,0.0
4,28,79,24,2019,1,5,6.0,0.83,223.0,3.0,71.0,959.0,107.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,28,79,24,2019,12,27,NaN,0.59,20.0,6.8,72.0,953.0,95.0,0.0
361,28,79,24,2019,12,28,NaN,0.67,344.0,5.8,68.0,953.0,98.0,0.4
362,28,79,24,2019,12,29,NaN,0.64,13.0,5.7,71.0,954.0,98.0,0.0
363,28,79,24,2019,12,30,NaN,0.66,125.0,4.9,79.0,955.0,95.0,0.0


## Export Preprocess dataset

In [77]:
# # Export daily data
weather_2019.to_csv('weather_daily_2019_24.csv', index=False, sep = ';')